In [1]:
import json
import os
import jsonschema
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.getenv('NB_STUDIO_API_KEY')
)

model = "meta-llama/Meta-Llama-3.1-8B-Instruct-fast"

class WorkHistory(BaseModel):
    company: str
    duration: float
    position: str

sample_json_schema = WorkHistory.schema()

messages = [{
        "role": "system",
        "content": "you are a helpful assistant"
    }, {
        "role":
        "user",
        "content":
        f"Give an example JSON for an employee profile that "
        f"fits this schema: {sample_json_schema}"
    }]

chat_completion = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=1000,
    tools=[{
        "type": "function",
        "function": {
            "name": "dummy_function_name",
            "description": "This is a dummy function",
            "parameters": sample_json_schema
        }
    }],
    tool_choice={
        "type": "function",
        "function": {
            "name": "dummy_function_name"
        }
    })
message = chat_completion.choices[0].message

json_string = message.tool_calls[0].function.arguments
json1 = json.loads(json_string)
jsonschema.validate(instance=json1, schema=sample_json_schema)

In [2]:
message.dict()

{'content': '',
 'refusal': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [{'id': 'chatcmpl-tool-b0d02469afc744feaeee8c8d1497d186',
   'function': {'arguments': '{"company": "Google", "duration": 5, "position": "Software Engineer"}',
    'name': 'dummy_function_name'},
   'type': 'function'}]}